In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('./wine.csv')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

정형 데이터의 끝판왕 = 앙상블 학습 알고리즘
: 더 좋은 결과를 도촐하기 위해 여러 개의 모델을 합쳐놓은 알고리즘

0. 앙상블 학습
- 정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘으로, 여러 개의 모델을 합쳐서 더 좋은 결과를 도출함.
- 주로 결정 트리를 기반으로 만들어져 있음.

1. 랜덤 포레스트
- 앙상블 학습의 대표주자로, 랜덤한 결정트리의 숲이라고 보면 됨.
- : 기본원리: A+B로 개별 트리에 무작위성을 부여해서 트리의 성능이 너무 강력해지는 것을 막음(=과대적합 방지)
- ->물론 개별트리의 성능은 떨어짐 but 그걸 여러 개 묶어서 일반화하면 높은 성능이 나오게 됨.

A: 훈련세트에 무작위성 주입
- 랜덤 포레스트의 속의 각 트리는 우리가 입력한 훈련데이터를 그대로 학습하지 않고, 훈련세트와 같은 크기의 부트스트랩 샘플(중복허용)을 만들어 학습함.
B: 특성 선택에 무작위성 주입
- 노드를 분할할 때도 모든 특성을 다 써서 최선의 분할을 하는 게 아니라, 루트(특성개수)개의 특성만 써서 분할하게 함.
C: 노드 분할에 무작위성 주입
- 랜덤 포레스트가 부트스트랩 샘플 사용했던 것과 달리, 엑스트라 트리는 우리가 입력한 훈련세트 전체를 그대로 사용함.
- 대신, 노드를 분할할 때 최선의 분할(불순도 차이가 가장 큰 분할)을 찾는게 아니라 그냥 무작위로 분할함.

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8903229806766861


In [3]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23183515 0.50059756 0.26756729]


In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8945545507023283


In [5]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


분할 자체를 무작위로 하다보니 랜덤 포레스트보다 무작위성이 더 크다. 그래서 랜덤포레스트보다는 트리 개수를 더 많이 해야 좋은 성능을 낸다고 알려져있다. 하지만 더 큰 무작위성 때문에 계산속도는 상대적으로 더 빨라진다는 장점도 있다.

In [6]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


그레이디언트 부스팅
- 깊이가 얖은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식의 앙상블 학습.
- (기본값 = 3 / 깊이가 얕으니까 역시 과적합을 방지할 수 있음)
- Gradient에서 유추할 수 있듯, 경사 하강법을 사용해 트리를 추가하는 거임.(분류: 로지스틱 손실함수/회귀: 평균제곱오차 함수)
**이전 트리의 손실을 보완하는 방향으로 트리 추가**
!경사하강법에서 손실함수의 낮은 곳을 잧아 조금씩 이동했던 것처럼!
- 깊이가 얕은 트리를 사용해서 트리의 성닝이 강력해지는 것을 막음(과적합 방지)
- 물론 처음에 score구해보면 성능이 많이 높지 않음 but 그건 트리 개수 점점 더 넣으면서 높이면 됨.(결정트리 개수 늘려도 과대적합에 강하다는 장점)
- 조금씩 이동하도록 학습률도 조절함 by learning_rate(기본값 = 0.1)

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [8]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [9]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15881044 0.67988912 0.16130044]


히스토그램 기반 GB
-정형 데이터를 다루는 머신러닝 알고리즘 중 가장 인기가 높음
-기본 매개번수에서도 안정적인 성능을 얻을 수 있을 정도로 괜찮은 모델

입력 특성을 256개의 구간으로 나눔
- 노드를 분할하기 전에, 훈련데이터를 256개의 구간으로 쪼갬 -> 특성의 범위가 짧게 끊어져있으니 최적의 분할을 매우 빠르게 찾을 수 있음


In [10]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [11]:
from sklearn.inspection import permutation_importance #특성 중요도

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [12]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1) #특성 중요도 확인
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [13]:
hgb.score(test_input, test_target)

0.8723076923076923

OOB 샘플(랜덤 포레스트 중)
: 랜덤포레스트에서 부트스트랩 샘플을 만들 때, 복원추출 과정에서 선택되지 않고 남은 샘플 -> 이 남은 샘플로 트리를 평가할 수 있음(like 검증세트)
- obb_score_: ood_score 옵션을 True로 하면 그걸로 평가한 점수가 여기에 저장됨.

subsample(그레이디언트 부스팅 중)
- subsample 매개변수를 사용하면 트리 훈련에 쓸 훈련세트의 비율을 정할 수 있음.
- 그런데 이걸 1보다 작은 겂으로 넣으면 훈련세트의 일부만 사용하게 됨. => 즉, 앞에서 배웠던 확률적 경사하강법이나 미니배치 경사하강법처럼 되는 거임.

일반 검증 - 교차 검증 - 앙상블 학습, 이렇게 3가지를 배웠다.
실제 사용 과정에서 헷갈리는 느낌이 들어 개괄하여 적어둔다⭐
.
먼저 가장 기초적인 일반검증은 원래
dt = DecisionTreeClassifier() 처럼 모델 객체 만들고
dt.fit(입력, 타깃) 해서 가지고 있는 데이터로 훈련한 뒤,
dt.score(입력, 타깃) 해서 그 성능을 평가할 수 있었다
.
근데 좀 더 발전된 교차검증으로 오면
dt = DecisionTreeClassifier() 해서 모델 객체 만들어 놓은 것에
cross_validate(dt, 입력, 타깃) 하면 알아서 fit하고 score까지 한 다음, 그 결과값을 (딕셔너리 형태로) 반환해준다
.
그리고 성능이 가장 뛰어난 앙상블은
처음에 dt = ___ 이 자리에 앙상블 모델만 넣어주면 되는 것이다!!
이후는 똑같이 cross_validate 해주는 것이 일반적인 듯하다 (당연히 교차검증이 더 효율적이니 교재에서도 그렇게 사용한 듯) 🆗

더 강력한 성능의 모델을 만들기 위해 "앙상블 학습"을 도입 → 랜덤 포레스트와 엑스트라 트리는 성능을 낮춘 결정트리들을 앙상블 : 안정적인 성능 + 특성 의존도 줄어듦 → 그레이디언트 부스팅은 경사하강법의 원리로 얕은 트리들을 추가하면서 높은 성능에 다가감 → 이것의 속도와 성능을 더 개선한 것이 히스토그램 기반 그레이디언트 부스팅이고, 훈련 데이터를 256개 구간으로 쪼개서 속도를 높임 + permutation_importance로 특성중요도 계산! → 그 외에도 XGBoost, LightGBM 등 많은 앙상블 알고리즘들이 있음. 